<a href="https://colab.research.google.com/github/UrayLaszlo/Airbnb_project/blob/main/nyc_airbnb_01_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount drive, create folders
from google.colab import drive
drive.mount('/content/drive')
PROJECT_ROOT = '/content/drive/MyDrive/ColabNotebooks/Airbnb_project/nyc_airbnb_da'
!mkdir -p $PROJECT_ROOT/data/raw
%cd $PROJECT_ROOT


Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/Airbnb_project/nyc_airbnb_da


In [3]:
# Install libraries
%%bash
pip install -q geopandas shap prophet xgboost lightgbm shapely contextily pyarrow

'# Install libraries\n%%bash\npip install -q geopandas shap prophet xgboost lightgbm shapely contextily pyarrow\n'

In [4]:
# Grab the latest NYC files
import re, requests, pandas as pd, io, gzip, pathlib, os, datetime as dt

CITY = "new-york-city"
BASE = "http://insideairbnb.com/get-the-data.html"
RAW_DIR = pathlib.Path("data/raw"); RAW_DIR.mkdir(parents=True, exist_ok=True)

page = requests.get(BASE).text
pattern = rf'https?://[^"]+{CITY}[^"]+\.csv\.gz'
links = re.findall(pattern, page)
latest_links = {
    pathlib.Path(url).stem.split('.')[0]: url        # → 'listings', 'calendar', 'reviews'
    for url in links
}

for name, url in latest_links.items():
    fn = RAW_DIR / f"{name}.csv.gz"
    if fn.exists():
        print(f"✓ {fn} already present")
        continue
    print(f"↓ downloading {url.split('/')[-1]}")
    r = requests.get(url, stream=True)
    with open(fn, 'wb') as f: f.write(r.content)

# Quick sanity peek
listings_head = pd.read_csv(RAW_DIR/'listings.csv.gz', nrows=5)
listings_head.head()


✓ data/raw/listings.csv.gz already present
✓ data/raw/calendar.csv.gz already present
✓ data/raw/reviews.csv.gz already present


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,36121,https://www.airbnb.com/rooms/36121,20250301213336,2025-03-03,city scrape,Lg Rm in Historic Prospect Heights,Cozy space share in the heart of a great neigh...,Full of tree-lined streets and beautiful brown...,https://a0.muscache.com/pictures/8776354/35b32...,62165,...,5.00,5.00,5.00,NaN,f,1,0,1,0,0.05
1,36647,https://www.airbnb.com/rooms/36647,20250301213336,2025-03-03,city scrape,"1 Bedroom & your own Bathroom, Elevator Apartment",Private bedroom with your own bathroom in a 2 ...,"Manhattan, SE corner of 2nd Ave/ E. 110th street",https://a0.muscache.com/pictures/miso/Hosting-...,157798,...,4.90,4.38,4.71,NaN,f,1,0,1,0,0.58
2,38663,https://www.airbnb.com/rooms/38663,20250301213336,2025-03-02,city scrape,Luxury Brownstone in Boerum Hill,"Beautiful, large home in great hipster neighbo...","diverse, lively, hip, cool: loaded with restau...",https://a0.muscache.com/pictures/miso/Hosting-...,165789,...,4.88,4.86,4.62,OSE-STRREG-0001784,f,1,0,1,0,0.28
3,38833,https://www.airbnb.com/rooms/38833,20250301213336,2025-03-03,city scrape,Spectacular West Harlem Garden Apt,This is a very large and unique space. An inc...,West Harlem is now packed with great restauran...,https://a0.muscache.com/pictures/7554f9e5-4ab0...,166532,...,4.96,4.79,4.82,OSE-STRREG-0000476,f,1,1,0,0,1.36
4,39282,https://www.airbnb.com/rooms/39282,20250301213336,2025-03-02,city scrape,“Work-from-home” from OUR home.,*Monthly Discount will automatically apply <br...,THE NEIGHBORHOOD:<br />Our apartment is locate...,https://a0.muscache.com/pictures/ef8f43ad-d967...,168525,...,4.88,4.85,4.78,OSE-STRREG-0001150,f,2,0,2,0,1.54
